In [1]:
# Import needed libraries
import pandas as pd
import math, datetime
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
import re
from sklearn.cluster import KMeans

style.use('ggplot')
%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df

,Page,Clicks,Impressions,CTR,Position
0,https://buffert.se/basta-aktierna-och-fonderna...,29585,331619,8.92%,8.86
1,https://buffert.se/loner-i-sverige/,19890,181742,10.94%,6.31
2,https://buffert.se/hur-mycket-far-jag-lana/,16497,362826,4.55%,26.61
3,https://buffert.se/basta-bankerna-privatpersoner/,13347,147683,9.04%,22.57
4,https://buffert.se/vaxla-pengar-utomlands-bill...,13300,341707,3.89%,14.74
...,...,...,...,...,...
904,https://buffert.se/author/svante-hansson/page/5/,0,1,0%,17.00
905,https://buffert.se/rantor/#Drojsmalsranta,0,1,0%,17.00
906,https://buffert.se/buffert-reser-till-japan-20...,0,1,0%,30.00
907,https://buffert.se/fackforbund/#Vanliga-fragor,0,1,0%,82.00


In [4]:
# Cleaning The Data
df['CTR'] = df['CTR'].apply(lambda x: float(x.replace('%', '')))

In [6]:
X = df[['Clicks', 'Impressions', 'Position']]
y = df.pop('CTR')

KeyError: 'CTR'

In [7]:
# Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)

In [6]:
# Standardise The Data 
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
scaler = scaler.fit(X_train)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [8]:
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns, index = X_train.index)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [9]:
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns, index = X_test.index)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [118]:
# Import different Algorithms to see differences between their predictions
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [119]:
Linear_regression = LinearRegression()

In [120]:
Linear_regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [121]:
Linear_regression.score(X_train, y_train)

0.2217570360002491

In [122]:
Linear_regression.score(X_test, y_test)

0.12411512924894186

In [125]:
RandomForestRegressorModel = RandomForestRegressor(n_estimators=50)
RandomForestRegressorModel.fit(X_train, y_train)
prediction_score = RandomForestRegressorModel.score(X_train, y_train)
test_score = RandomForestRegressorModel.score(X_test, y_test)

print(prediction_score, test_score)

0.9720072617992609 0.7151004129111413


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
pipeline = make_pipeline(RandomForestRegressor(n_estimators=200))
 
# Declare hyperparameters to tune
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [5, 3, None]}
 
# Tune model using cross-validation pipeline
RandomForestRegressorModel = GridSearchCV(pipeline, hyperparameters, cv=5)
 
RandomForestRegressorModel.fit(X_train, y_train)
prediction_score = RandomForestRegressorModel.score(X_train, y_train)
print("The score of prediction for RandomForestRegressorModel is: {}".format(prediction_score))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-e77c553189f0>", line 13, in <module>
    RandomForestRegressorModel.fit(X_train, y_train)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py", line 722, in fit
    self._run_search(evaluate_candidates)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py", line 1191, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py", line 711, in evaluate_candidates
    cv.split(X, y, groups)))
  File "/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 920, in __call__
    while self.dispatch_one_batch(iterator):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.p

KeyboardInterrupt: 

### Let's Create A Function That Combines The Predictions For X_train, X_test and also contains the index number so that we can match it back up with the original dataframe.

In [25]:
X_list = [X_train, X_test]
y_list = y_train, y_test

RandomForestRegressorModel = RandomForestRegressor(n_estimators=50)

data_dict = {
    
    'Indexes': [],
    'Predictions': []    
    
}

for x, y in zip(X_list, y_list):
    
    indexes = list(x.index)
    RandomForestRegressorModel.fit(X_train, y_train)
    predictions = RandomForestRegressorModel.predict(x)
    
    data_dict['Indexes'].extend(indexes)
    data_dict['Predictions'].extend(predictions)

------------------------------------------------------------------------------------------------------------------------

In [30]:
data_dict

{'Indexes': [595,
  140,
  173,
  6,
  585,
  405,
  73,
  622,
  398,
  468,
  145,
  234,
  220,
  380,
  436,
  132,
  526,
  332,
  185,
  41,
  722,
  108,
  432,
  56,
  720,
  857,
  512,
  24,
  360,
  688,
  447,
  51,
  781,
  832,
  819,
  264,
  449,
  621,
  611,
  361,
  18,
  651,
  583,
  575,
  83,
  61,
  272,
  285,
  368,
  535,
  12,
  182,
  394,
  891,
  223,
  521,
  670,
  789,
  636,
  531,
  554,
  176,
  761,
  456,
  836,
  163,
  248,
  578,
  801,
  74,
  113,
  518,
  506,
  104,
  114,
  779,
  92,
  513,
  89,
  336,
  897,
  833,
  844,
  551,
  766,
  94,
  11,
  514,
  43,
  42,
  329,
  167,
  793,
  545,
  862,
  100,
  178,
  591,
  798,
  601,
  177,
  596,
  616,
  721,
  630,
  383,
  257,
  451,
  335,
  559,
  15,
  3,
  422,
  256,
  898,
  542,
  425,
  393,
  222,
  179,
  467,
  705,
  324,
  538,
  9,
  249,
  22,
  356,
  221,
  658,
  780,
  892,
  536,
  340,
  431,
  756,
  527,
  203,
  237,
  93,
  500,
  837,
  567,
  284,
  184,

------------------------------------------------------------------------------------------------------------------------------

### Merging X_train + X_test with Y_train + Y_test

In [70]:
merged_X = pd.concat([X_train, X_test])
merged_Y = pd.concat([y_train, y_test])

In [71]:
merged_df = pd.concat([merged_X, merged_Y], axis = 1)

In [73]:
merged_df.head(12)

,Clicks,Impressions,Position,CTR
595,-0.133582,-0.314960,2.022484,0.00
140,-0.123166,0.099501,-0.561524,0.15
173,-0.128374,0.105610,-0.715761,0.07
6,5.562623,3.851732,-0.269951,8.29
585,-0.133582,-0.314679,0.371825,0.00
405,-0.133582,-0.241618,-0.637058,0.00
73,-0.058937,0.002530,-0.496554,1.42
622,-0.133582,-0.315627,1.500083,0.00
398,-0.133582,-0.227363,-0.708895,0.00
468,-0.133582,-0.300741,-0.668751,0.00


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Predicted Data For Click Through Ratio's For All Pages

In [82]:
predicted_data = pd.DataFrame(data_dict)
predicted_data.set_index(predicted_data['Indexes'], inplace = True)
predicted_data.drop(columns=['Indexes'], inplace = True)
predicted_data.rename(columns={"Predictions": "Predicted_CTR_%"}, inplace = True)

In [84]:
predicted_data.head(6)

,Predicted_CTR_%
Indexes,
595,0.0000
140,0.1642
173,0.0716
6,6.8456
585,0.0000
405,0.0000


----------------------------------------------------------------------------------------------------------------------------------------

### Merging The Original Dataframe to the predicted CTR (%).

In [86]:
final_df = pd.concat([merged_df, predicted_data], axis=1)

In [87]:
final_df

,Clicks,Impressions,Position,CTR,Predicted_CTR_%
595,-0.133582,-0.314960,2.022484,0.00,0.0000
140,-0.123166,0.099501,-0.561524,0.15,0.1642
173,-0.128374,0.105610,-0.715761,0.07,0.0716
6,5.562623,3.851732,-0.269951,8.29,6.8456
585,-0.133582,-0.314679,0.371825,0.00,0.0000
...,...,...,...,...,...
835,-0.133582,-0.317242,-0.314321,0.00,0.0000
193,-0.130110,0.083983,-0.706254,0.05,0.0550
316,-0.133003,-0.228662,-0.663468,0.04,0.0672
581,-0.133582,-0.314504,-0.694633,0.00,0.0000


In [91]:
final_df['CTR_Difference'] = final_df['Predicted_CTR_%'] - final_df['CTR']

In [ ]:
### Now let's subset on the data and remove any CTR's that were 0.00

In [96]:
final_df = final_df[final_df['CTR'] != 0.00]

,Clicks,Impressions,Position,CTR,Predicted_CTR_%,CTR_Difference
140,-0.123166,0.099501,-0.561524,0.15,0.1642,0.0142
173,-0.128374,0.105610,-0.715761,0.07,0.0716,0.0016
6,5.562623,3.851732,-0.269951,8.29,6.8456,-1.4444
73,-0.058937,0.002530,-0.496554,1.42,1.5926,0.1726
145,-0.123745,-0.231015,-0.700971,0.69,0.6940,0.0040
...,...,...,...,...,...,...
164,-0.127217,-0.228311,1.867718,0.43,0.3766,-0.0534
28,0.515660,0.300499,-0.278931,6.38,5.3966,-0.9834
193,-0.130110,0.083983,-0.706254,0.05,0.0550,0.0050
316,-0.133003,-0.228662,-0.663468,0.04,0.0672,0.0272


In [101]:
final_df.sort_values(by='CTR_Difference', ascending=False)

,Clicks,Impressions,Position,CTR,Predicted_CTR_%,CTR_Difference
44,0.118130,0.697265,-0.255161,1.51,4.7008,3.1908
8,2.818098,4.847947,-0.530887,3.47,5.5242,2.0542
4,7.562428,11.679617,-0.099339,3.89,5.5130,1.6230
2,9.412364,12.421082,0.527647,4.55,6.1140,1.5640
350,-0.133003,-0.312081,0.465846,0.67,2.1184,1.4484
...,...,...,...,...,...,...
5,5.599078,3.320639,-0.486518,9.56,6.3590,-3.2010
40,0.196826,-0.201488,-0.559939,17.30,13.9258,-3.3742
30,0.424813,0.072502,0.185366,8.69,5.2566,-3.4334
79,-0.067038,-0.276446,0.021621,9.87,5.0988,-4.7712


In [132]:
final_df = final_df[['CTR', 'Predicted_CTR_%', 'CTR_Difference']]

In [133]:
original_data = df[['Page', 'Clicks', 'Impressions', 'Position']]

In [135]:
results = pd.merge(original_data, final_df, left_index=True, right_index=True)

In [137]:
results.to_csv('results.csv')

In [138]:
results

,Page,Clicks,Impressions,Position,CTR,Predicted_CTR_%,CTR_Difference
0,https://buffert.se/basta-aktierna-och-fonderna...,29585,331619,8.86,8.92,8.2136,-0.7064
1,https://buffert.se/loner-i-sverige/,19890,181742,6.31,10.94,9.5772,-1.3628
2,https://buffert.se/hur-mycket-far-jag-lana/,16497,362826,26.61,4.55,6.1140,1.5640
3,https://buffert.se/basta-bankerna-privatpersoner/,13347,147683,22.57,9.04,7.5192,-1.5208
4,https://buffert.se/vaxla-pengar-utomlands-bill...,13300,341707,14.74,3.89,5.5130,1.6230
...,...,...,...,...,...,...,...
904,https://buffert.se/author/svante-hansson/page/5/,0,1,17.00,0.00,0.0000,0.0000
905,https://buffert.se/rantor/#Drojsmalsranta,0,1,17.00,0.00,0.0000,0.0000
906,https://buffert.se/buffert-reser-till-japan-20...,0,1,30.00,0.00,0.0000,0.0000
907,https://buffert.se/fackforbund/#Vanliga-fragor,0,1,82.00,0.00,0.0000,0.0000
